# 策略回测方法

## 📋 概述

策略回测是量化交易中验证策略有效性的关键步骤，通过在历史数据上模拟交易来评估策略的表现。本章节将详细介绍策略回测的方法、回测指标、常见问题以及如何避免回测陷阱。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际代码示例理解策略回测的方法。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib akshare

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 回测的基本概念

#### 1.1 什么是回测

**回测（Backtesting）**：
- 在历史数据上模拟交易
- 评估策略的表现
- 验证策略的有效性

**回测的目的**：
- 验证策略逻辑
- 评估策略收益和风险
- 优化策略参数
- 发现策略问题

#### 1.2 回测的步骤

1. **数据准备**：获取历史数据，清洗数据
2. **策略实现**：编写策略代码
3. **信号生成**：根据策略生成交易信号
4. **模拟交易**：按照信号执行交易
5. **性能评估**：计算收益、风险等指标
6. **结果分析**：分析回测结果，发现问题

让我们创建一个简单的回测引擎：


In [ ]:
# 回测引擎类
class BacktestEngine:
    """回测引擎"""
    
    def __init__(self, initial_capital=1000000, commission=0.001):
        """
        初始化回测引擎
        
        参数:
        initial_capital: 初始资金
        commission: 手续费率
        """
        self.initial_capital = initial_capital
        self.capital = initial_capital
        self.commission = commission
        self.positions = {}  # {symbol: quantity}
        self.trades = []  # 交易记录
        self.equity_curve = []  # 权益曲线
        self.daily_returns = []  # 每日收益率
    
    def execute_trade(self, symbol, action, quantity, price, date):
        """
        执行交易
        
        参数:
        symbol: 股票代码
        action: 'buy' or 'sell'
        quantity: 交易数量
        price: 交易价格
        date: 交易日期
        """
        if action == 'buy':
            cost = quantity * price * (1 + self.commission)
            if self.capital >= cost:
                self.capital -= cost
                if symbol not in self.positions:
                    self.positions[symbol] = 0
                self.positions[symbol] += quantity
                self.trades.append({
                    'date': date,
                    'symbol': symbol,
                    'action': 'buy',
                    'quantity': quantity,
                    'price': price,
                    'cost': cost
                })
                return True
        elif action == 'sell':
            if symbol in self.positions and self.positions[symbol] >= quantity:
                revenue = quantity * price * (1 - self.commission)
                self.capital += revenue
                self.positions[symbol] -= quantity
                if self.positions[symbol] == 0:
                    del self.positions[symbol]
                self.trades.append({
                    'date': date,
                    'symbol': symbol,
                    'action': 'sell',
                    'quantity': quantity,
                    'price': price,
                    'revenue': revenue
                })
                return True
        return False
    
    def update_portfolio_value(self, date, prices):
        """
        更新组合价值
        
        参数:
        date: 日期
        prices: {symbol: price} 当前价格字典
        """
        portfolio_value = self.capital
        for symbol, quantity in self.positions.items():
            if symbol in prices:
                portfolio_value += quantity * prices[symbol]
        
        self.equity_curve.append({
            'date': date,
            'value': portfolio_value,
            'capital': self.capital,
            'position_value': portfolio_value - self.capital
        })
        
        # 计算每日收益率
        if len(self.equity_curve) > 1:
            prev_value = self.equity_curve[-2]['value']
            daily_return = (portfolio_value - prev_value) / prev_value
            self.daily_returns.append(daily_return)
    
    def calculate_metrics(self):
        """计算回测指标"""
        if len(self.equity_curve) == 0:
            return {}
        
        equity_df = pd.DataFrame(self.equity_curve)
        final_value = equity_df['value'].iloc[-1]
        total_return = (final_value - self.initial_capital) / self.initial_capital
        
        # 计算年化收益率
        days = (equity_df['date'].iloc[-1] - equity_df['date'].iloc[0]).days
        years = days / 365.0
        annual_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
        
        # 计算波动率
        returns = pd.Series(self.daily_returns)
        volatility = returns.std() * np.sqrt(252)  # 年化波动率
        
        # 计算夏普比率（假设无风险利率为3%）
        risk_free_rate = 0.03
        sharpe_ratio = (annual_return - risk_free_rate) / volatility if volatility > 0 else 0
        
        # 计算最大回撤
        equity_values = equity_df['value'].values
        cumulative_max = np.maximum.accumulate(equity_values)
        drawdown = (equity_values - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()
        
        # 计算胜率
        if len(self.trades) > 0:
            trades_df = pd.DataFrame(self.trades)
            buy_trades = trades_df[trades_df['action'] == 'buy']
            sell_trades = trades_df[trades_df['action'] == 'sell']
            # 简化计算，实际需要配对买卖交易
            win_rate = 0.5  # 占位符
        else:
            win_rate = 0
        
        return {
            '总收益率': total_return * 100,
            '年化收益率': annual_return * 100,
            '年化波动率': volatility * 100,
            '夏普比率': sharpe_ratio,
            '最大回撤': max_drawdown * 100,
            '交易次数': len(self.trades),
            '最终价值': final_value
        }
    
    def run_backtest(self, data, signals, symbol='stock'):
        """
        运行回测
        
        参数:
        data: 价格数据（DataFrame，包含Close列）
        signals: 交易信号（Series，1买入，-1卖出，0持仓）
        symbol: 股票代码
        """
        for i in range(len(data)):
            date = data.index[i]
            price = data['Close'].iloc[i]
            signal = signals.iloc[i] if i < len(signals) else 0
            
            # 根据信号执行交易
            if signal > 0:  # 买入信号
                # 计算买入数量（使用全部资金）
                quantity = int(self.capital / (price * (1 + self.commission)) / 100) * 100  # 按手买入
                if quantity > 0:
                    self.execute_trade(symbol, 'buy', quantity, price, date)
            elif signal < 0:  # 卖出信号
                # 卖出全部持仓
                if symbol in self.positions:
                    quantity = self.positions[symbol]
                    self.execute_trade(symbol, 'sell', quantity, price, date)
            
            # 更新组合价值
            self.update_portfolio_value(date, {symbol: price})
        
        return self.calculate_metrics()

print("回测引擎类定义完成！")


In [ ]:
# 获取数据
try:
    import akshare as ak
    stock_data = ak.stock_zh_a_hist(symbol="000001", period="daily", 
                                     start_date="20230101", end_date="20241231", 
                                     adjust="qfq")
    stock_data.columns = ['日期', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率']
    stock_data['日期'] = pd.to_datetime(stock_data['日期'])
    stock_data = stock_data.set_index('日期')
    stock_data = stock_data[['开盘', '最高', '最低', '收盘', '成交量']]
    stock_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    stock_data = stock_data.sort_index()
    print("数据获取成功！")
except Exception as e:
    print(f"数据获取失败：{e}，使用模拟数据")
    dates = pd.date_range(start='2023-01-01', end='2024-12-31', freq='D')
    dates = dates[dates.weekday < 5]
    np.random.seed(42)
    prices = 10 + np.cumsum(np.random.randn(len(dates)) * 0.1)
    stock_data = pd.DataFrame({
        'Open': prices + np.random.randn(len(dates)) * 0.05,
        'High': prices + np.abs(np.random.randn(len(dates)) * 0.1),
        'Low': prices - np.abs(np.random.randn(len(dates)) * 0.1),
        'Close': prices + np.random.randn(len(dates)) * 0.05,
        'Volume': np.random.randint(1000000, 10000000, len(dates))
    }, index=dates)
    stock_data['High'] = stock_data[['Open', 'Close', 'High']].max(axis=1)
    stock_data['Low'] = stock_data[['Open', 'Close', 'Low']].min(axis=1)

# 生成交易信号（双均线策略）
short_window = 5
long_window = 20
stock_data['MA_Short'] = stock_data['Close'].rolling(window=short_window).mean()
stock_data['MA_Long'] = stock_data['Close'].rolling(window=long_window).mean()
stock_data['Signal'] = 0
stock_data.loc[stock_data['MA_Short'] > stock_data['MA_Long'], 'Signal'] = 1
stock_data.loc[stock_data['MA_Short'] < stock_data['MA_Long'], 'Signal'] = -1

# 运行回测
backtest = BacktestEngine(initial_capital=1000000, commission=0.001)
metrics = backtest.run_backtest(stock_data, stock_data['Signal'], symbol='000001')

# 显示回测结果
print("\n回测结果：")
for key, value in metrics.items():
    if isinstance(value, float):
        if '率' in key or '比率' in key:
            print(f"{key}: {value:.2f}%")
        else:
            print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")


### 3. 回测结果可视化

让我们可视化回测结果：


In [ ]:
# 可视化回测结果
fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True,
                          gridspec_kw={'height_ratios': [3, 2, 1]})

equity_df = pd.DataFrame(backtest.equity_curve)
recent_data = stock_data.tail(300)
recent_equity = equity_df.tail(300)

# 1. 价格和信号
axes[0].plot(recent_data.index, recent_data['Close'], label='收盘价', linewidth=1.5, color='black')
axes[0].plot(recent_data.index, recent_data['MA_Short'], label=f'MA{short_window}', linewidth=1.5, color='blue')
axes[0].plot(recent_data.index, recent_data['MA_Long'], label=f'MA{long_window}', linewidth=1.5, color='red')

# 标注买卖点
buy_signals = recent_data[recent_data['Signal'] == 1]
sell_signals = recent_data[recent_data['Signal'] == -1]
axes[0].scatter(buy_signals.index, buy_signals['Close'] * 0.98, 
               marker='^', color='green', s=100, label='买入', zorder=5)
axes[0].scatter(sell_signals.index, sell_signals['Close'] * 1.02, 
               marker='v', color='red', s=100, label='卖出', zorder=5)

axes[0].set_title('股票价格与交易信号', fontsize=14, fontweight='bold')
axes[0].set_ylabel('价格')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. 权益曲线
axes[1].plot(recent_equity['date'], recent_equity['value'], 
            label='组合价值', linewidth=2, color='green')
axes[1].axhline(y=backtest.initial_capital, color='red', 
               linestyle='--', label='初始资金')
axes[1].set_title('回测权益曲线', fontsize=14, fontweight='bold')
axes[1].set_ylabel('组合价值')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 3. 回撤曲线
equity_values = equity_df['value'].values
cumulative_max = np.maximum.accumulate(equity_values)
drawdown = (equity_values - cumulative_max) / cumulative_max * 100
recent_drawdown = drawdown[-300:]

axes[2].fill_between(recent_equity['date'], recent_drawdown, 0, 
                     color='red', alpha=0.3, label='回撤')
axes[2].plot(recent_equity['date'], recent_drawdown, 
            linewidth=1.5, color='red')
axes[2].set_title('回撤曲线', fontsize=14, fontweight='bold')
axes[2].set_xlabel('日期')
axes[2].set_ylabel('回撤 (%)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### 4. 回测常见问题

#### 4.1 过拟合

**问题**：策略在历史数据上表现很好，但在新数据上表现差

**原因**：
- 过度优化参数
- 使用未来信息
- 样本量不足

**解决方法**：
- 样本外测试
- 交叉验证
- 避免过度优化

#### 4.2 未来函数

**问题**：使用了未来信息，导致回测结果不真实

**常见情况**：
- 使用当日收盘价生成当日信号
- 使用未来数据计算指标

**解决方法**：
- 使用前一日数据生成当日信号
- 确保数据时间对齐

#### 4.3 幸存者偏差

**问题**：只回测了存续的股票，忽略了退市股票

**解决方法**：
- 使用全市场数据
- 考虑退市股票的影响

## 💡 关键要点总结

1. **回测步骤**：数据准备→策略实现→信号生成→模拟交易→性能评估
2. **回测指标**：收益率、夏普比率、最大回撤、胜率等
3. **常见问题**：过拟合、未来函数、幸存者偏差
4. **注意事项**：样本外测试、避免过度优化、确保数据真实性

## 🛠️ 工具与软件

### 回测平台

- **聚宽（JoinQuant）**：国内量化平台，回测功能强大，无需梯子
- **米筐（RiceQuant）**：国内量化平台，无需梯子
- **Backtrader**：Python回测框架，开源免费
- **Zipline**：Python回测框架，开源

### 注意事项

- ✅ **数据质量**：确保数据准确性和完整性
- ✅ **样本外测试**：必须在样本外数据上验证
- ✅ **避免过拟合**：不要过度优化参数
- ⚠️ **未来函数**：确保不使用未来信息
- ⚠️ **交易成本**：考虑手续费、滑点等成本

## 🔗 相关知识点

- [策略设计原理](./策略设计原理.ipynb)
- [策略优化方法](./策略优化方法.md)
- [量化交易系统架构](../01_量化交易基础/量化交易系统架构.ipynb)

## 📚 拓展阅读

- 《量化交易：如何建立自己的算法交易》- 回测方法
- 《Python量化交易实战》- 实战案例
- 各量化平台文档 - 了解平台回测功能
